# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
! pip install yfinance numpy pandas scipy XlsxWriter ipython

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [4]:
my_columns = ['Ticker',
              'Stock Price',
              'P/E Ratio',
              'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [5]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    price = data['currentPrice']
    try:
        peRatio = data['currentPrice'] / data['trailingEps']
    except:
        peRatio = np.NaN
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [price],
        'P/E Ratio': [peRatio],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price  P/E Ratio Number of Shares to Buy
0        A     108.9950  28.607612                     N/A
1      AAL      12.8564   3.437540                     N/A
2      AAP      51.8000   8.946459                     N/A
3     AAPL     173.2830  29.025628                     N/A
4     ABBV     147.5500  30.297741                     N/A
..     ...          ...        ...                     ...
498    YUM     122.9400  24.736419                     N/A
499    ZBH     109.1450  45.667364                     N/A
500   ZBRA     220.2600  17.892770                     N/A
501   ZION      33.4060   5.819861                     N/A
502    ZTS     170.6900  35.784067                     N/A

[503 rows x 4 columns]


In [6]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,A,108.9950,28.607612,N/A
1,AAL,12.8564,3.437540,N/A
2,AAP,51.8000,8.946459,N/A
3,AAPL,173.2830,29.025628,N/A
4,ABBV,147.5500,30.297741,N/A
...,...,...,...,...
498,YUM,122.9400,24.736419,N/A
499,ZBH,109.1450,45.667364,N/A
500,ZBRA,220.2600,17.892770,N/A
501,ZION,33.4060,5.819861,N/A


In [7]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
80,CAT,260.83,NaN,N/A


Filling the `NULL` cells of the `P/E Ratio` column with mean value

In [8]:
final_dataframe['P/E Ratio'].fillna(final_dataframe['P/E Ratio'].mean(), inplace=True)

In [9]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy


In [10]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,A,108.9950,28.607612,N/A
1,AAL,12.8564,3.437540,N/A
2,AAP,51.8000,8.946459,N/A
3,AAPL,173.2830,29.025628,N/A
4,ABBV,147.5500,30.297741,N/A
...,...,...,...,...
498,YUM,122.9400,24.736419,N/A
499,ZBH,109.1450,45.667364,N/A
500,ZBRA,220.2600,17.892770,N/A
501,ZION,33.4060,5.819861,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll consider only those stocks whose `P/E Ratio` is positive.

In [11]:
final_dataframe.sort_values('P/E Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['P/E Ratio'] > 0]

In [12]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
1,AAL,12.8564,3.437540,N/A
100,CMA,39.7900,4.153445,N/A
206,GM,30.5250,4.411127,N/A
166,EQT,41.0400,4.495071,N/A
468,VLO,128.1400,4.581337,N/A
...,...,...,...,...
473,VTR,40.4900,184.045455,N/A
357,PANW,235.0200,185.055118,N/A
431,TAP,60.4900,302.450000,N/A
482,WELL,82.3300,304.925926,N/A


In [13]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [14]:
final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,AAL,12.8564,3.437540,N/A
1,CMA,39.7900,4.153445,N/A
2,GM,30.5250,4.411127,N/A
3,EQT,41.0400,4.495071,N/A
4,VLO,128.1400,4.581337,N/A
5,ALB,152.7400,4.592303,N/A
6,PSX,109.6900,4.949910,N/A
7,UAL,41.4250,5.133209,N/A
8,SYF,28.6400,5.245421,N/A
9,MOS,33.8900,5.353870,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [15]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [17]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,AAL,12.8564,3.437540,15556
1,CMA,39.7900,4.153445,5026
2,GM,30.5250,4.411127,6552
3,EQT,41.0400,4.495071,4873
4,VLO,128.1400,4.581337,1560
5,ALB,152.7400,4.592303,1309
6,PSX,109.6900,4.949910,1823
7,UAL,41.4250,5.133209,4828
8,SYF,28.6400,5.245421,6983
9,MOS,33.8900,5.353870,5901


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

In [18]:
rv_columns = ['Ticker',
              'Stock Price',
              'Number of Shares to Buy',
              'Price-to-Earning Ratio',
              'PE Percentile',
              'Price-to-Book Value',
              'PB Percentile',
              'Price-to-Sales Ratio',
              'PS Percentile',
              'EV/EBITDA',
              'EV/EBITDA Percentile',
              'EV/GP',
              'EV/GP Percentile',
              'RV Score']

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [19]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    financials = yf.Ticker(stock).financials

    try:
        price = data['currentPrice']
    except:
        price = np.NaN

    try:
        eps = data['trailingEps']
    except:
        eps = 1

    try:
        peRatio = price / eps
    except:
        peRatio = np.NaN

    try:
        pbRatio = data['priceToBook']
    except:
        pbRatio = np.NaN

    try:
        marketCap = data['marketCap']
    except:
        marketCap = np.NaN

    try:
        outstandingShares = data['sharesOutstanding']
    except:
        outstandingShares = 1

    try:
        psRatio = marketCap / (outstandingShares * eps)
    except:
        psRatio = np.NaN

    try:
        debt = data['totalDebt']
    except:
        debt = np.NaN

    try:
        cash = data['totalCash']
    except:
        cash = np.NaN

    try:
        ebitda = data['ebitda']
    except:
        ebitda = 1

    try:
        evToEBIDTA = (marketCap + debt - cash) / ebitda
    except:
        evToEBIDTA = np.NaN

    try:
        grossProfit = financials.loc['Gross Profit'][0]
    except:
        grossProfit = 1

    try:
        evToGP = (marketCap + debt - cash) / grossProfit
    except:
        evToGP = np.NaN

    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [price],
        'Number of Shares to Buy': ['N/A'],
        'Price-to-Earning Ratio': [peRatio],
        'PE Percentile': ['N/A'],
        'Price-to-Book Value': [pbRatio],
        'PB Percentile': ['N/A'],
        'Price-to-Sales Ratio': [psRatio],
        'PS Percentile': ['N/A'],
        'EV/EBITDA': [evToEBIDTA],
        'EV/EBITDA Percentile': ['N/A'],
        'EV/GP': [evToGP],
        'EV/GP Percentile': ['N/A'],
        'RV Score': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    rv_dataframe = pd.concat([rv_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(rv_dataframe)

    Ticker  Stock Price Number of Shares to Buy  Price-to-Earning Ratio  \
0        A      109.310                     N/A               28.690289   
1      AAL       12.810                     N/A                3.425134   
2      AAP       51.900                     N/A                8.963731   
3     AAPL      173.500                     N/A               29.061977   
4     ABBV      147.740                     N/A               30.336756   
..     ...          ...                     ...                     ...   
498    YUM      122.945                     N/A               24.737425   
499    ZBH      108.760                     N/A               45.506276   
500   ZBRA      219.950                     N/A               17.867587   
501   ZION       33.400                     N/A                5.818815   
502    ZTS      171.000                     N/A               35.849057   

    PE Percentile  Price-to-Book Value PB Percentile  Price-to-Sales Ratio  \
0             N/A    

In [20]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,109.310,N/A,28.690289,N/A,5.757400,N/A,28.690287,N/A,1.900548e+01,N/A,9.032962e+00,N/A,N/A
1,AAL,12.810,N/A,3.425134,N/A,NaN,N/A,3.425134,N/A,5.204278e+00,N/A,4.303593e+00,N/A,N/A
2,AAP,51.900,N/A,8.963731,N/A,1.125667,N/A,8.963730,N/A,8.499322e+00,N/A,1.492409e+00,N/A,N/A
3,AAPL,173.500,N/A,29.061977,N/A,45.041540,N/A,29.061978,N/A,2.226039e+01,N/A,1.615704e+01,N/A,N/A
4,ABBV,147.740,N/A,30.336756,N/A,20.266119,N/A,30.336757,N/A,1.073419e+01,N/A,7.711503e+00,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.945,N/A,24.737425,N/A,NaN,N/A,24.737425,N/A,1.940987e+01,N/A,1.399835e+01,N/A,N/A
499,ZBH,108.760,N/A,45.506276,N/A,1.835612,N/A,45.506275,N/A,1.200298e+01,N/A,5.756569e+00,N/A,N/A
500,ZBRA,219.950,N/A,17.867587,N/A,3.773633,N/A,17.867586,N/A,1.226272e+01,N/A,5.192028e+00,N/A,N/A
501,ZION,33.400,N/A,5.818815,N/A,1.021688,N/A,5.818815,N/A,8.067043e+09,N/A,8.067043e+09,N/A,N/A


In [21]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
1,AAL,12.8100,N/A,3.425134,N/A,NaN,N/A,3.425134e+00,N/A,5.204278,N/A,4.303593e+00,N/A,N/A
52,AZO,2495.8450,N/A,19.034815,N/A,NaN,N/A,1.903482e+01,N/A,13.977439,N/A,6.050087e+00,N/A,N/A
53,BA,186.7025,N/A,-25.436308,N/A,NaN,N/A,-2.543631e+01,N/A,140.582030,N/A,4.315411e+01,N/A,N/A
57,BBWI,32.1500,N/A,10.935374,N/A,NaN,N/A,1.093537e+01,N/A,8.460326,N/A,3.812995e+00,N/A,N/A
66,BKNG,2996.4100,N/A,26.970387,N/A,NaN,N/A,2.697039e+01,N/A,16.909395,N/A,1.062925e+11,N/A,N/A
78,CAH,89.7200,N/A,89.720000,N/A,NaN,N/A,8.972000e+01,N/A,9.475580,N/A,3.386396e+00,N/A,N/A
80,CAT,261.5100,N/A,261.510000,N/A,NaN,N/A,1.350296e+11,N/A,NaN,N/A,NaN,N/A,N/A
98,CL,69.7300,N/A,38.955307,N/A,NaN,N/A,3.895531e+01,N/A,16.598692,N/A,6.396197e+00,N/A,N/A
142,DPZ,365.4400,N/A,27.642965,N/A,NaN,N/A,2.764296e+01,N/A,21.051325,N/A,1.090289e+01,N/A,N/A
171,ETSY,61.9100,N/A,-11.401473,N/A,NaN,N/A,-1.140147e+01,N/A,20.663170,N/A,4.901403e+00,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [22]:
for column in ['Price-to-Earning Ratio', 'Price-to-Book Value', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [23]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

In [24]:
metrics = {
    'Price-to-Earning Ratio': 'PE Percentile',
    'Price-to-Book Value': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

In [25]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,109.310,N/A,28.690289,0.717694,5.757400,0.675944,28.690287,0.713718,1.900548e+01,0.735586,9.032962e+00,0.477137,N/A
1,AAL,12.810,N/A,3.425134,0.081511,10.257804,0.846918,3.425134,0.081511,5.204278e+00,0.059642,4.303593e+00,0.15507,N/A
2,AAP,51.900,N/A,8.963731,0.178926,1.125667,0.085487,8.963730,0.17495,8.499322e+00,0.16501,1.492409e+00,0.019881,N/A
3,AAPL,173.500,N/A,29.061977,0.725646,45.041540,0.976143,29.061978,0.719682,2.226039e+01,0.813121,1.615704e+01,0.77336,N/A
4,ABBV,147.740,N/A,30.336756,0.751491,20.266119,0.952286,30.336757,0.739563,1.073419e+01,0.294235,7.711503e+00,0.389662,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.945,N/A,24.737425,0.612326,10.257804,0.846918,24.737425,0.60835,1.940987e+01,0.747515,1.399835e+01,0.723658,N/A
499,ZBH,108.760,N/A,45.506276,0.882704,1.835612,0.264414,45.506275,0.87674,1.200298e+01,0.393638,5.756569e+00,0.258449,N/A
500,ZBRA,219.950,N/A,17.867587,0.431412,3.773633,0.544732,17.867586,0.423459,1.226272e+01,0.405567,5.192028e+00,0.208748,N/A
501,ZION,33.400,N/A,5.818815,0.107356,1.021688,0.067594,5.818815,0.107356,8.067043e+09,0.960239,8.067043e+09,0.910537,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [26]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

In [27]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,109.310,N/A,28.690289,0.717694,5.757400,0.675944,28.690287,0.713718,1.900548e+01,0.735586,9.032962e+00,0.477137,0.664016
1,AAL,12.810,N/A,3.425134,0.081511,10.257804,0.846918,3.425134,0.081511,5.204278e+00,0.059642,4.303593e+00,0.15507,0.24493
2,AAP,51.900,N/A,8.963731,0.178926,1.125667,0.085487,8.963730,0.17495,8.499322e+00,0.16501,1.492409e+00,0.019881,0.124851
3,AAPL,173.500,N/A,29.061977,0.725646,45.041540,0.976143,29.061978,0.719682,2.226039e+01,0.813121,1.615704e+01,0.77336,0.80159
4,ABBV,147.740,N/A,30.336756,0.751491,20.266119,0.952286,30.336757,0.739563,1.073419e+01,0.294235,7.711503e+00,0.389662,0.625447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.945,N/A,24.737425,0.612326,10.257804,0.846918,24.737425,0.60835,1.940987e+01,0.747515,1.399835e+01,0.723658,0.707753
499,ZBH,108.760,N/A,45.506276,0.882704,1.835612,0.264414,45.506275,0.87674,1.200298e+01,0.393638,5.756569e+00,0.258449,0.535189
500,ZBRA,219.950,N/A,17.867587,0.431412,3.773633,0.544732,17.867586,0.423459,1.226272e+01,0.405567,5.192028e+00,0.208748,0.402783
501,ZION,33.400,N/A,5.818815,0.107356,1.021688,0.067594,5.818815,0.107356,8.067043e+09,0.960239,8.067043e+09,0.910537,0.430616


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [28]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)

In [29]:
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)

In [30]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MHK,80.6100,N/A,-16.828810,0.037773,0.625538,0.017893,-16.828810,0.037773,5.828358e+00,0.085487,2.728304e+00,0.055666,0.046918
1,C,39.4800,N/A,6.266667,0.119284,0.403380,0.005964,6.266667,0.119284,-3.310204e+11,0.001988,-3.310204e+11,0.001988,0.049702
2,MOS,33.7750,N/A,5.335703,0.099404,0.904574,0.04175,5.335703,0.099404,3.830500e+00,0.031809,2.507481e+00,0.047714,0.064016
3,EQT,41.0300,N/A,4.493976,0.087475,1.235955,0.111332,4.493976,0.087475,3.069075e+00,0.025845,2.523358e+00,0.049702,0.072366
4,T,14.6650,N/A,-12.642241,0.04175,1.028906,0.073559,-12.754428,0.04175,6.102095e+00,0.093439,3.789527e+00,0.131213,0.076342
5,PARA,11.6150,N/A,-4.683468,0.069583,0.349176,0.003976,-5.066364,0.067594,9.438663e+00,0.210736,2.240412e+00,0.039761,0.07833
6,STT,63.6550,N/A,8.613667,0.168986,0.922416,0.047714,8.613667,0.166998,-3.813297e+10,0.011928,-3.813297e+10,0.011928,0.081511
7,WRK,36.2850,N/A,-6.537838,0.063618,0.929265,0.05169,-6.537838,0.063618,6.367142e+00,0.101392,4.661298e+00,0.166998,0.089463
8,IVZ,13.4250,N/A,9.589286,0.188867,0.532126,0.007952,9.589286,0.186879,5.287631e+00,0.06163,1.500147e+00,0.021869,0.093439
9,VTRS,9.7650,N/A,6.642857,0.125249,0.561788,0.011928,6.642858,0.125249,5.820450e+00,0.081511,4.594276e+00,0.163022,0.101392


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [31]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [32]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
position_size

200000.0

In [33]:
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[row, 'Stock Price'])

In [34]:
rv_dataframe

,Ticker,Stock Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Value,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MHK,80.6100,15556,-16.828810,0.037773,0.625538,0.017893,-16.828810,0.037773,5.828358e+00,0.085487,2.728304e+00,0.055666,0.046918
1,C,39.4800,5026,6.266667,0.119284,0.403380,0.005964,6.266667,0.119284,-3.310204e+11,0.001988,-3.310204e+11,0.001988,0.049702
2,MOS,33.7750,6552,5.335703,0.099404,0.904574,0.04175,5.335703,0.099404,3.830500e+00,0.031809,2.507481e+00,0.047714,0.064016
3,EQT,41.0300,4873,4.493976,0.087475,1.235955,0.111332,4.493976,0.087475,3.069075e+00,0.025845,2.523358e+00,0.049702,0.072366
4,T,14.6650,1560,-12.642241,0.04175,1.028906,0.073559,-12.754428,0.04175,6.102095e+00,0.093439,3.789527e+00,0.131213,0.076342
5,PARA,11.6150,1309,-4.683468,0.069583,0.349176,0.003976,-5.066364,0.067594,9.438663e+00,0.210736,2.240412e+00,0.039761,0.07833
6,STT,63.6550,1823,8.613667,0.168986,0.922416,0.047714,8.613667,0.166998,-3.813297e+10,0.011928,-3.813297e+10,0.011928,0.081511
7,WRK,36.2850,4828,-6.537838,0.063618,0.929265,0.05169,-6.537838,0.063618,6.367142e+00,0.101392,4.661298e+00,0.166998,0.089463
8,IVZ,13.4250,6983,9.589286,0.188867,0.532126,0.007952,9.589286,0.186879,5.287631e+00,0.06163,1.500147e+00,0.021869,0.093439
9,VTRS,9.7650,5901,6.642857,0.125249,0.561788,0.011928,6.642858,0.125249,5.820450e+00,0.081511,4.594276e+00,0.163022,0.101392


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [35]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Formatting Our Excel Output

In [36]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_template = writer.book.add_format(
    {
        'num_format': '0.0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [37]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Stock Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earning Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Value', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [38]:
writer.close()